# Assignment 10

### 1.复习上课内容

### 2. 回答一下理论题目

#### 1. What is independent assumption in Naive bayes ?

独立指统计意义上的独立，指一个特征或者单词出现的可能性与它和其他单词相邻没有关系。

#### 2. What is MAP(maximum a posterior) and ML(maximum likelihood) ?

MAP：最大化后验概率是贝叶斯派模型参数估计的方法，是根据已知样本，来通过调整模型参数使得模型能够产生该数据样本的概率最大，只不过对于模型参数有了一个先验假设，即模型参数可能满足某种分布，不再一味地依赖数据样例

ML：极大似然估计是频率学派模型参数估计方法，要最大化该事件发生的可能性，含义是根据已知样本，希望通过调整模型参数来使得模型能够最大化样本情况出现的概率，本质意义上是我们在很多参数中选择一个能使数据出现概率最大的参数。

#### 3. What is support vector in SVM?

将数据集分隔开来的平面为分隔超平面，离分隔超平面最近的那些点称为支持向量。

#### 4. What is the intuition behind SVM ?

尽可能将数据分为正样本和负样本，通过确定他们位于超平面哪一侧来对数据完成分类，其中要最大化支持向量到分隔面的距离来求解分隔超平面，可引入拉格朗日乘子法计算。

#### 5. Shortly describ what 'random' means in random forest ?

假设有N个样本，M个特征，对于每棵树都有放回的随机抽取训练样本，随机抽取n个样本作为训练集，再有放回的随机选取m个特征作为这棵树的分枝的依据。所以随机有两层含义：随机抽取样本和随机选取特征。

#### 6. What cariterion does XGBoost use to find the best split point in a tree ?

特征预排序+缓存：XGBoost在训练之前，预先对每个特征按照特征值大小进行排序，然后保存为block结构，后面的迭代中会重复地使用这个结构，使计算量大大减小。

分位点近似法：对每个特征按照特征值排序后，采用类似分位点选取的方式，仅仅选出常数个特征值作为该特征的候选分割点，在寻找该特征的最佳分割点时，从候选分割点中选出最优的一个。

并行查找：由于各个特性已预先存储为block结构，XGBoost支持利用多个线程并行地计算每个特征的最佳分割点，这不仅大大提升了结点的分裂速度，也极利于大规模训练集的适应性扩展。

### 3. Practial part

##### Problem description: In this part you are going to build a classifier to detect if a piece of news is published by the Xinhua news agency (新华社）.

#### Hints:

###### 1. Firstly, you have to come up with a way to represent the news. (Vectorize the sentence, you can find different ways to do so online)  

###### 2. Secondly,  pick a machine learning algorithm that you think is suitable for this task

### 方法：贝叶斯分类器

In [1]:
import pandas as pd
import numpy as np
filename = '新华社数据.csv'
file = pd.read_csv(filename,encoding='gb18030')


In [2]:
file.head(50)

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm
3,89614,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....
4,89613,胡淑丽_MN7479,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",...",44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,http://news.163.com/17/0618/00/CN617P3Q0001875...
5,89612,张怡,中国证券报?中证网,受到A股被纳入MSCI指数的利好消息刺激，A股市场从周三开始再度上演龙马行情，周四上午金...,"{""type"":""市场"",""site"":""中证网"",""commentNum"":""0"",""jo...",金融股一枝独秀 配置价值犹存,http://www.cs.com.cn/gppd/201706/t20170623_533...
6,89611,NaN,威锋网@http://www.feng.com/,虽然至今夏普智能手机在市场上无法排得上号，已经完全没落，并于 2013 年退出中国市场，但是...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""21""...",配骁龙660 全面屏鼻祖夏普新机酝酿中,http://www.cnbeta.com/articles/tech/623603.htm
7,89610,申玉彬 整理,中国证券报?中证网,沙漠雄鹰：震荡有利消化套牢筹码\r\n 周四开盘上证50在银行券商大蓝筹带动下一度涨近...,"{""type"":""市场"",""site"":""中证网"",""commentNum"":""0"",""jo...",博友早评：震荡有利消化套牢筹码,http://www.cs.com.cn/gppd/201706/t20170623_533...
8,89609,李杭_BJS4645,荆楚网-楚天都市报,（原标题：武汉警方一下子抓了808人，还都是俊男靓女！原来他们每天偷偷摸摸干这事！）\r\n...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""1600""...",武汉千余警察出动 抓获808名俊男靓女全是诈骗犯,http://news.163.com/17/0614/14/CMT9N8G80001899...
9,89608,吴瞬,中国证券报?中证网,6月21日，A股纳入MSCI指数尘埃落定，但当天被寄予厚望的券商股并未扛起反弹大旗。22...,"{""type"":""市场"",""site"":""中证网"",""commentNum"":""0"",""jo...",纳入MSCI指数 A股长期配置价值提升,http://www.cs.com.cn/gppd/201706/t20170623_533...


In [167]:
import jieba
import re
def cut(string):
    return jieba.lcut(string)
def token(string):
    return re.findall('\w+',string)
TOKEN =[]
content = file['content'].tolist()
for i,c in enumerate(content[10500:11500]):
#for i,c in enumerate(content[11000:12000]):
    if i % 1000 == 0: print(i)
    c = ''.join(token(str(c)))
    TOKEN.append(cut(c))
print(len(TOKEN))


0
1000


In [168]:
source = file['source'].tolist()
print(len(source))
#classes = np.zeros(len(source))
classes = [0]*1000
for i,c in enumerate(source[10500:11500]):

    if '新华社' in str(c):
        
        classes[i]=1
#print(classes[39])
print(sum(classes))

89611
390


#### 将列表中单词转为向量

In [7]:
def Create_wordVec(dataset):
    word_set = set([])
    for doc in dataset:
        word_set = word_set | set(doc) # 通过对两个集合取并，找出所有非重复的单词
    return list(word_set)

In [67]:
def Words2Vec(wordList, input_set):
    '''
    @wordList：为前一个函数的输出值（包含单词）
    @input_set：输入需要分类的集合
    函数输出：包含0，1的布尔型向量（对应Wordlist中的单词出现与否）
    '''
    return_vec = [0] * len(wordList) 
    # 创建与词汇表等长的列表向量
    for word in input_set:
        if word in wordList:

            return_vec[wordList.index(word)]+= 1 # 出现的单词赋1
        else: print("the word %s is not in list" % word)
    return return_vec

In [169]:
vocaList = Create_wordVec(TOKEN)
vocaList

['茶',
 '岗位津贴',
 '492',
 '顾虑',
 '连续',
 '国际航空',
 '487',
 'nnn20132014',
 '特稿',
 '潜质',
 '南沙',
 '应自',
 '太长',
 '工信部',
 '华说',
 '谢美琴',
 '地势',
 '战力',
 '初中文化',
 '六市',
 '使然',
 '二七路',
 '恶势力',
 '保持',
 '被查',
 '事件',
 '26',
 '座椅',
 '港深',
 '刚开',
 '铺装',
 '求职',
 '运营',
 '漂亮',
 '不折不扣',
 '汤凯锋',
 '做长',
 '学前班',
 '自视甚高',
 '情节',
 '两翼',
 '红牛',
 '共见',
 '营收',
 '第二季度',
 '农业',
 '处处长',
 '打针',
 '筹备',
 '升至',
 '理论知识',
 '该片',
 '亲和力',
 '添',
 '而菲永',
 '串串',
 '735',
 '选材',
 '固定',
 '潮雄帝',
 '护栏',
 '出手',
 '布言布语',
 '王超',
 '1927',
 '协调局',
 '下转',
 '电表',
 '漯河市',
 '奋赛',
 '多地',
 '曹剑杰',
 '中行',
 '2010',
 '能力',
 '廖某',
 '随便',
 '国投',
 '充足',
 '到位',
 '埋头苦干',
 '溜冰场',
 '三水区',
 '障碍赛',
 '出于',
 '配音',
 '带到',
 '四方',
 '这步田地',
 '民事纠纷',
 '苏黎世',
 '南航',
 '诺言',
 '工作力度',
 '批次',
 '不受',
 '立义',
 '顺',
 '坐在',
 '四大',
 '赤字',
 '本事',
 'Facebook',
 '尴尬',
 '诊治',
 '奥列格',
 '贸易额',
 '两',
 '用手',
 '高送',
 '天津',
 '城市交通',
 '重点项目',
 '右马索',
 '王保建',
 '地震',
 '外媒',
 '三座',
 '高额',
 '驻外',
 '暨南大学',
 '案情',
 '村貌',
 '一排',
 '开放型',
 '一道道',
 '克洛德',
 '帮助',
 '涨薪',
 '问责',
 '摸准',
 '春耕生产',
 '

In [96]:
VocaList = Create_wordVec(TOKEN)
VocaList

['楼板',
 '马蹄',
 '梅西于',
 '投骑',
 '6766',
 '难以逾越',
 '许多',
 '张家',
 '起王',
 '全球网',
 '分间',
 '人身自由权',
 '田为',
 '玛沁县',
 '下莫迪',
 '瞎眼',
 '驾时',
 '江森',
 '广西壮族自治区',
 '西康',
 '3271',
 '后汉书',
 '一不留神',
 '索降',
 '内分泌科',
 '提朝',
 '网签',
 '种得',
 '平原县',
 '测图',
 '张克军',
 '放射性',
 '极具',
 '住房供给',
 '下课后',
 '罗瑛',
 '农宣',
 '本格拉',
 '拍花',
 '董本洪',
 '抗炎药',
 'AB',
 '王家卫',
 '蔡林格',
 '乔治亚',
 'TT',
 '甲类',
 '体育频道',
 '添绿',
 '研修生',
 '觉完',
 '毫无根据',
 '施瓦布',
 '南开大学',
 '帅歌',
 '十七年',
 '残留物',
 '贺利氏',
 '诸城市',
 '边防',
 '陈冠华',
 '扶重',
 '禄勇良',
 '永寿',
 'TechCrunch',
 '马罗',
 '遭南',
 'M777',
 '顶配',
 '出史亮',
 '韩永进',
 '支固息',
 '宣传画',
 '省图书馆',
 '远差',
 '店成',
 '情窦初开',
 '花花世界',
 '横加',
 '恭王府',
 '左七',
 '凝固',
 '干酪',
 '齐麟',
 '西南面',
 '象甲',
 '宋长志',
 '王晓辉',
 '管昕据',
 '易加',
 'P75I',
 '判监',
 '真刀',
 '题莎娃',
 '上清',
 '天外来客',
 '非生产',
 '直取',
 '只租不卖',
 '管理体制',
 '战柯洁',
 '置疑',
 '辉于',
 '76n',
 '严抓',
 '明智之举',
 '飞毛腿',
 '明风烈',
 '奇思妙想',
 '宋彦麟',
 '数百盏',
 '乡西',
 '能源危机',
 '330430',
 '1579',
 '飞驰而来',
 '幅员',
 '徐悲鸿',
 '博亚',
 '罗曼诺夫',
 '多室',
 '直选',
 '174658',
 '严处',
 '405259',
 '吃皇粮',
 '

In [10]:
print(len(VocaList))

NameError: name 'VocaList' is not defined

In [170]:
print(len(vocaList))

25886


In [147]:
import numpy
from numpy import *


def train_bayes(train_matrix, train_class):
    '''
    利用NumPy数组计算p(wi|c1)
    词条 属于类1的概率Prob_x = p(c1) 
    因为是二分类所以属于类0概率 =1-p(c1)
    '''
    num_docs = len(train_matrix)       #50

    num_word = len(train_matrix[0])    #47216

    # 获取输入文档（句子）数以及向量的长度
    Prob_x = sum(train_class)/ float(num_docs) #

    Prob_num_0 = np.ones(num_word) # 创建一个长度为词条向量等长的列表
    Prob_num_1 = np.ones(num_word) 
    # 为避免0概率使得最终乘积为0，使用拉普拉斯平滑（加入常数lamda，此处为1）
    Prob_denom_0 = 2.0
    Prob_denom_1 = 2.0
    
    for i in range(num_docs):
    # 统计类别为1的词条向量中出现的所有词条的总数
            # 即统计类1所有文档中出现单词的数目
        if train_class[i] == 1:
 
            Prob_num_1 += train_matrix[i]
            Prob_denom_1 += sum(train_matrix[i])
        else:
            Prob_num_0 += train_matrix[i]
            Prob_denom_0 += sum(train_matrix[i])
    #print('Prob_num_1:',Prob_num_1)        #[1. 1. 1. ... 1. 1. 1.]
    #print('Prob_denom_1:',Prob_denom_1)    #48

    p1_vec = log(Prob_num_1 / Prob_denom_1) #[-3.87120101 -3.87120101 -3.87120101 ... -3.87120101 -3.87120101 -3.87120101]
    p0_vec = log(Prob_num_0 / Prob_denom_0)
    #print('p1_vec:',p1_vec)

    # 将结果取自然对数，避免下溢出，即太多很小的数相乘造成的影响
    return p1_vec, p0_vec, Prob_x
    # p1概率实际上等于[p(w1|c1),p(w2|c1), ... p(wn|c1)]组成的向量
    # 后续计算中，会基于条件独立性求出p(W|c1)

In [148]:


def classify_bayes(test_vec, p0, p1, p_pos):
    '''
    @vec2Classify:待测试分类的词条向量
    @p0:类别0所有文档中各个词条出现的频数p(wi|c0)
    @p1:类别1所有文档中各个词条出现的频数p(wi|c1)
    @p_pos:类别为1的文档占文档总数比例
    '''
    
    p1 = sum(test_vec * p1) + log( p_pos) #新华社
    p0 = sum(test_vec * p0) + log( 1.0 - p_pos) #others
    # 原公式为乘积P(w|c)P(c)，log取对数后乘积变为相加
    #print(p1,p0)
    if p1 > p0:
        return 1
    else:
        return 0
    


In [184]:
from sklearn.model_selection import KFold
def Xinhua_filter():

    #将所有新闻中出现的字符串构建成字符串列表
    
    #构建一个大小为1000的整数列表和一个空列表
    trainingSet = list(range(1000)); testSet = []
    #随机选取1~1000中的100个数，作为索引，构建测试集

#新建两个列表
    trainMat=[];trainClasses=[]
    fold = KFold(10,shuffle=True) #创建10折交叉验证

    for trainingSet, testSet in fold.split(TOKEN):

    
    #遍历训练集中的吗每个字符串列表

        for docIndex in trainingSet:
        #将字符串列表转为词条向量，然后添加到训练矩阵中
            trainMat.append(Words2Vec(vocaList, TOKEN[docIndex]))
        #将该新闻的类标签存入训练类标签列表中
            trainClasses.append(classes[docIndex])
    #计算贝叶斯函数需要的概率值并返回
        p1V,p0V,pXinhua = train_bayes(array(trainMat), array(trainClasses))
        errorCount = 0
    #遍历测试集中的字符串列表
   
        for docIndex in testSet:
        #同样将测试集中的字符串列表转为词条向量
            
            wordVector = Words2Vec(vocaList,TOKEN[docIndex])
        # print(wordVector)
        #对测试集中字符串向量进行预测分类，分类结果不等于实际结果
            if classify_bayes(pd.array(wordVector),p0V,p1V,pXinhua)!=classes[docIndex]:  #不是新华社的文章判断为新华社，+1
                errorCount += 1
        print('the error rate is:',float(errorCount)/ len(testSet))
        print('errorCount:',errorCount)

In [187]:
Xinhua_filter() #输出交叉验证结果

the error rate is: 0.04
errorCount: 4
the error rate is: 0.02
errorCount: 2
the error rate is: 0.0
errorCount: 0
the error rate is: 0.0
errorCount: 0
the error rate is: 0.01
errorCount: 1
the error rate is: 0.0
errorCount: 0
the error rate is: 0.0
errorCount: 0
the error rate is: 0.0
errorCount: 0
the error rate is: 0.01
errorCount: 1
the error rate is: 0.01
errorCount: 1


### Congratulations! You have completed all assignments in this week. The question below is optional. If you still have time, why don't try it out.

## Option:

#### Try differnt machine learning algorithms with different combinations of parameters in the practical part, and compare their performances (Better use some visualization techiniques).